In [3]:
import chardet

with open('hebrew names.txt', 'rb') as file:
    raw_data = file.read()
    detected = chardet.detect(raw_data)
    encoding = detected['encoding']

words = open('hebrew names.txt', 'r', encoding=encoding).read().splitlines()

In [4]:
del words[0]

In [255]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

{' ': 1,
 '"': 2,
 "'": 3,
 '?': 4,
 'm': 5,
 'א': 6,
 'ב': 7,
 'ג': 8,
 'ד': 9,
 'ה': 10,
 'ו': 11,
 'ז': 12,
 'ח': 13,
 'ט': 14,
 'י': 15,
 'ך': 16,
 'כ': 17,
 'ל': 18,
 'ם': 19,
 'מ': 20,
 'ן': 21,
 'נ': 22,
 'ס': 23,
 'ע': 24,
 'ף': 25,
 'פ': 26,
 'ץ': 27,
 'צ': 28,
 'ק': 29,
 'ר': 30,
 'ש': 31,
 'ת': 32,
 '.': 0}

In [256]:
import torch

In [155]:
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

In [156]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=33).float()

In [169]:
g = torch.Generator().manual_seed(120)
W = torch.randn((33, 33), generator=g, requires_grad=True)
logits = xenc @ W # log-counts
counts = logits.exp().float() # equivalent N
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean()
num = xs.nelement()

In [259]:
for k in range(1):
  # forward pass
  xenc = F.one_hot(xs, num_classes=33).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  # update
  W.data += -100 * W.grad

2.6337528228759766


In [261]:
for i in range(20):
  out = []
  ix = 0
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=33).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    if ix == 0:
        break
    out.append(itos[ix])
  print(''.join(out))

מה
נתיור
יה
ד
נבי
פינו
מי
אט
חזנאשרונה
אה
בהא
סה
ג'ירצו
ק
מויאיה
שיגזא
מסניוצם
צנטף
ג'
מג'ו
